<a href="https://colab.research.google.com/github/SMN-SENAMHI/ADAPTACION/blob/main/CMIP6-REGIONAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q xarray zarr fsspec==2025.3.0 numcodecs requests==2.32.4 aiohttp > /dev/null 2>&1
!apt-get install -qq -y libproj-dev proj-data proj-bin libgeos-dev > /dev/null 2>&1
!pip install -q cartopy shapely pyproj > /dev/null 2>&1



In [2]:
#@title Importar librerías
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gpd
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import requests, os, warnings
#import warnings
from cartopy.io import DownloadWarning
warnings.filterwarnings("ignore", category=DownloadWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
# ==================================================
# Script: Información general del dataset DestinE
# ==================================================
import xarray as xr
import pandas as pd

# -----------------------------
# 1. Abrir dataset (ya debes tener tu token configurado en .netrc)
# -----------------------------
url = "https://cacheb.dcms.destine.eu/d1-climate-dt/ScenarioMIP-SSP3-7.0-IFS-NEMO-0001-high-sfc-v0.zarr"

ds = xr.open_dataset(
    url,
    engine="zarr",
    chunks={},
    storage_options={"client_kwargs": {"trust_env": True}}
)

print("✅ Dataset abierto correctamente\n")

# -----------------------------
# 2. Información temporal
# -----------------------------
fechas = pd.to_datetime(ds.time.values)

anio_inicio = fechas.min().year
anio_fin    = fechas.max().year
n_pasos     = len(fechas)

# calcular paso temporal en horas
delta_horas = (fechas[1] - fechas[0]).astype('timedelta64[h]').item()

if delta_horas == 1:
    frecuencia = "horaria"
elif delta_horas == 6:
    frecuencia = "6-horaria"
elif 23 <= delta_horas <= 25:
    frecuencia = "diaria"
elif 28*24 <= delta_horas <= 31*24:
    frecuencia = "mensual"
else:
    frecuencia = f"≈ cada {delta_horas} horas"

print("📅 Información temporal del dataset:")
print(f"Año inicial: {anio_inicio}")
print(f"Año final:   {anio_fin}")
print(f"Total de pasos de tiempo: {n_pasos}")
print(f"Frecuencia temporal: {frecuencia}\n")

# -----------------------------
# 3. Información espacial
# -----------------------------
lat_min, lat_max = ds.latitude.values.min(), ds.latitude.values.max()
lon_min, lon_max = ds.longitude.values.min(), ds.longitude.values.max()

resol_deg = 360 / ds.dims["longitude"]
resol_km  = 111 * resol_deg  # 1° ~ 111 km

print("🌍 Información espacial:")
print(f"Latitud:  {lat_min:.2f} a {lat_max:.2f}")
print(f"Longitud: {lon_min:.2f} a {lon_max:.2f}")
print(f"Número de puntos: {ds.dims['latitude']} x {ds.dims['longitude']}")
print(f"Resolución aproximada: {resol_deg:.3f}° ≈ {resol_km:.1f} km\n")

# -----------------------------
# 4. Variables disponibles
# -----------------------------
variables = []
for var in ds.data_vars:
    units = ds[var].attrs.get("units", "sin unidades")
    long_name = ds[var].attrs.get("long_name", "sin descripción")
    variables.append([var, units, long_name])

df_vars = pd.DataFrame(variables, columns=["Variable", "Unidades", "Descripción"])
print("📑 Variables disponibles:\n")
print(df_vars.to_string(index=False))

# Guardar en CSV
df_vars.to_csv("variables_destine.csv", index=False, encoding="utf-8")
print("\n✅ Catálogo de variables guardado en: variables_destine.csv")


ClientResponseError: 401, message='Unauthorized', url='https://cacheb.dcms.destine.eu/d1-climate-dt/ScenarioMIP-SSP3-7.0-IFS-NEMO-0001-high-sfc-v0.zarr/.zgroup'

In [ ]:
# ==============================================
# Guardar token en Colab (.netrc)
# ==============================================

#Reemplaza con tu token segun usuario registrado
#TOKEN = "edh_pat_cfdba8b50c493e84dfc676f5f1efed1794dde11c40cd21237e54c21da72c2f9378c792defeb8e2ca4638ac478df26ab8"
TOKEN = "edh_pat_fcb2e9f52c35ff7fd7cc2f2d14317393c9e299c8c1c1f81a94216161433a0943f554036a62b849c1af67f0c0400c31e3"

# Crear el archivo
with open("/root/.netrc", "w") as f:
    f.write("machine data.earthdatahub.destine.eu\n")
    f.write(f"  password {TOKEN}\n")

# Permisos
!chmod 600 /root/.netrc
